# Olypmic Games Analysis

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

## Data from Kaggle: 120 years of Olympic history - athletes and results
Source: https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results \
by Randi H Griffin, PhD - 
http://www.randigriffin.com

In [2]:
olympic = pd.read_csv('athlete_events.csv')
olympic.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
olympic.shape

(271116, 15)

In [70]:
olympic['City'].nunique()

42

In [21]:
singapore = olympic[olympic['NOC'].str.startswith('SG')]
singapore.shape

(349, 15)

In [15]:
singapore.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
578,332,Zarinah Abdullah,F,21.0,NaN,NaN,Singapore,SGP,1992 Summer,1992,Summer,Barcelona,Badminton,Badminton Women's Singles,NaN
579,332,Zarinah Abdullah,F,25.0,NaN,NaN,Singapore,SGP,1996 Summer,1996,Summer,Atlanta,Badminton,Badminton Women's Singles,NaN
2719,1515,Saiyidah Aisyah Mohamed Rafa'ee,F,28.0,173.0,68.0,Singapore,SGP,2016 Summer,2016,Summer,Rio de Janeiro,Rowing,Rowing Women's Single Sculls,NaN
7659,4264,Ang Peng Siong,M,21.0,180.0,75.0,Singapore,SGP,1984 Summer,1984,Summer,Los Angeles,Swimming,Swimming Men's 100 metres Freestyle,NaN
7660,4264,Ang Peng Siong,M,21.0,180.0,75.0,Singapore,SGP,1984 Summer,1984,Summer,Los Angeles,Swimming,Swimming Men's 4 x 100 metres Freestyle Relay,NaN


In [22]:
singapore.tail()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
268752,134454,Zhang Guirong,F,26.0,182.0,95.0,Singapore,SGP,2004 Summer,2004,Summer,Athina,Athletics,Athletics Women's Shot Put,NaN
268753,134454,Zhang Guirong,F,30.0,182.0,95.0,Singapore,SGP,2008 Summer,2008,Summer,Beijing,Athletics,Athletics Women's Shot Put,NaN
268993,134587,Zhang Xueling,F,21.0,163.0,53.0,Singapore,SGP,2004 Summer,2004,Summer,Athina,Table Tennis,Table Tennis Women's Singles,NaN
268994,134587,Zhang Xueling,F,21.0,163.0,53.0,Singapore-1,SGP,2004 Summer,2004,Summer,Athina,Table Tennis,Table Tennis Women's Doubles,NaN
269455,134826,Zhou Yihan,F,22.0,168.0,54.0,Singapore,SGP,2016 Summer,2016,Summer,Rio de Janeiro,Table Tennis,Table Tennis Women's Team,NaN


In [24]:
singapore['Team'].value_counts()

Singapore       338
Rika II           4
June Climene      3
Singapore-1       2
Singapore-2       2
Name: Team, dtype: int64

In [26]:
singapore[singapore['Team'] != 'Singapore']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
44376,22885,James Bryden Cooke,M,25.0,182.0,78.0,June Climene,SGP,1960 Summer,1960,Summer,Roma,Sailing,Sailing Mixed Three Person Keelboat,NaN
60185,30818,Thomas Kevin Durcan,M,39.0,184.0,86.0,June Climene,SGP,1960 Summer,1960,Summer,Roma,Sailing,Sailing Mixed Three Person Keelboat,NaN
80986,41124,Kenneth Dunstan Golding,M,NaN,NaN,NaN,Rika II,SGP,1956 Summer,1956,Summer,Melbourne,Sailing,Sailing Mixed Three Person Keelboat,NaN
96679,48948,Robert Ho,M,34.0,NaN,NaN,Rika II,SGP,1956 Summer,1956,Summer,Melbourne,Sailing,Sailing Mixed Three Person Keelboat,NaN
97496,49348,"Edward Gilbert ""Ned"" Holiday",M,56.0,171.0,83.0,Rika II,SGP,1956 Summer,1956,Summer,Melbourne,Sailing,Sailing Mixed Three Person Keelboat,NaN
97497,49348,"Edward Gilbert ""Ned"" Holiday",M,59.0,171.0,83.0,June Climene,SGP,1960 Summer,1960,Summer,Roma,Sailing,Sailing Mixed Three Person Keelboat,NaN
108582,54908,Jing Jun Hong,F,35.0,163.0,58.0,Singapore-2,SGP,2004 Summer,2004,Summer,Athina,Table Tennis,Table Tennis Women's Doubles,NaN
109582,55409,Keith Littlewood Johnson,M,59.0,NaN,NaN,Rika II,SGP,1956 Summer,1956,Summer,Melbourne,Sailing,Sailing Mixed Three Person Keelboat,NaN
138093,69392,Li Jia Wei,F,23.0,170.0,60.0,Singapore-2,SGP,2004 Summer,2004,Summer,Athina,Table Tennis,Table Tennis Women's Doubles,NaN
235895,118283,"Paey Fern ""Sharon"" Tan",F,30.0,160.0,43.0,Singapore-1,SGP,2004 Summer,2004,Summer,Athina,Table Tennis,Table Tennis Women's Doubles,NaN


In [59]:
noc = pd.read_csv('datasets_31029_40943_noc_regions.csv')
noc.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [60]:
noc.shape

(230, 3)

In [61]:
sgp = {'NOC':'SGP', 'region':'Singapore', 'notes':np.NaN}

In [62]:
noc = noc.append(sgp, ignore_index=True)

In [63]:
noc.shape

(231, 3)

In [64]:
noc.tail()

,NOC,region,notes
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN
229,ZIM,Zimbabwe,NaN
230,SGP,Singapore,NaN


In [65]:
noc.to_csv('noc_regions_RB.csv', index=False)

In [66]:
check = pd.read_csv('noc_regions_RB.csv')
check.tail()

,NOC,region,notes
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN
229,ZIM,Zimbabwe,NaN
230,SGP,Singapore,NaN


## The Dataset from Kaggle contains:
- "Art Competitions" like painting, sculpting, etc. from the first third of Olympic history. The modern Olympics, however, has decided that the art events are not legitimate.
- Medals won by athletes. If a football or hockey team wins the gold medal that counts as 1 gold for their country, however, as many gold medals for the athletes as many they are in the team.
- Revoked medals, for example, due to doping.
- Does not contain the 2018 Winter Olympics held in Pyeongchang, South Korea \
\
**For these reasons there is a significant difference between the medals won by athletes and by countries**

## Scraping the All-time Olympic Games medal table from Wikipedia
Source: https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table

In [166]:
url = 'https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
table = soup.body.find_all('table', {'class':'wikitable sortable'})[0]
rows = table.find_all('tr')
rows = [row.text.strip().split('\n') for row in rows]
data = rows[2:-1]
df = pd.DataFrame(data)
df = df.drop(columns=[1,2,7,8,13,14])
colnames = ['Country', 'Gold_Summer', 'Silver_Summer', 'Bronze_Summer', 'Total_Summer', 'Gold_Winter', 'Silver_Winter', 'Bronze_Winter', 'Total_Winter', 'Gold_Total', 'Silver_Total', 'Bronze_Total', 'Grand_Total']
df.columns = colnames
df.head()

,Country,Gold_Summer,Silver_Summer,Bronze_Summer,Total_Summer,Gold_Winter,Silver_Winter,Bronze_Winter,Total_Winter,Gold_Total,Silver_Total,Bronze_Total,Grand_Total
0,Afghanistan (AFG),0,0,2,2,0,0,0,0,0,0,2,2
1,Algeria (ALG),5,4,8,17,0,0,0,0,5,4,8,17
2,Argentina (ARG),21,25,28,74,0,0,0,0,21,25,28,74
3,Armenia (ARM),2,6,6,14,0,0,0,0,2,6,6,14
4,Australasia (ANZ) [ANZ],3,4,5,12,0,0,0,0,3,4,5,12


In [167]:
#pattern1 = r'\([A-Z]+\)'
#pattern2 = r'\[[[A-Z]+\]'
pattern1 = r'\(\w+\)'
pattern2 = r'\[\w+\]'

In [168]:
df['Country'] = df['Country'].str.replace(pattern1, '')

In [169]:
df['Country'] = df['Country'].str.replace(pattern2, '')

In [170]:
df['Country'] = df['Country'].str.strip()

In [171]:
df.head()

,Country,Gold_Summer,Silver_Summer,Bronze_Summer,Total_Summer,Gold_Winter,Silver_Winter,Bronze_Winter,Total_Winter,Gold_Total,Silver_Total,Bronze_Total,Grand_Total
0,Afghanistan,0,0,2,2,0,0,0,0,0,0,2,2
1,Algeria,5,4,8,17,0,0,0,0,5,4,8,17
2,Argentina,21,25,28,74,0,0,0,0,21,25,28,74
3,Armenia,2,6,6,14,0,0,0,0,2,6,6,14
4,Australasia,3,4,5,12,0,0,0,0,3,4,5,12


In [172]:
df[df['Country'].str.startswith('R')]

,Country,Gold_Summer,Silver_Summer,Bronze_Summer,Total_Summer,Gold_Winter,Silver_Winter,Bronze_Winter,Total_Winter,Gold_Total,Silver_Total,Bronze_Total,Grand_Total
106,Romania,89,95,122,306,0,0,1,1,89,95,123,307
107,Russia,148,126,152,426,47,38,35,120,195,164,187,546
108,Russian Empire,1,4,3,8,0,0,0,0,1,4,3,8


In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country        152 non-null    object
 1   Gold_Summer    152 non-null    object
 2   Silver_Summer  152 non-null    object
 3   Bronze_Summer  152 non-null    object
 4   Total_Summer   152 non-null    object
 5   Gold_Winter    152 non-null    object
 6   Silver_Winter  152 non-null    object
 7   Bronze_Winter  152 non-null    object
 8   Total_Winter   152 non-null    object
 9   Gold_Total     152 non-null    object
 10  Silver_Total   152 non-null    object
 11  Bronze_Total   152 non-null    object
 12  Grand_Total    152 non-null    object
dtypes: object(13)
memory usage: 15.6+ KB


In [174]:
#df['Grand_Total'] = df['Grand_Total'].str.replace(',', '')
#df['Grand_Total'] = df['Grand_Total'].astype(int)

In [175]:
def change_type(col):
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].astype(int)

In [176]:
#change_type('Gold_Winter')

In [177]:
lst = list(df.columns)
lst[1:]

['Gold_Summer',
 'Silver_Summer',
 'Bronze_Summer',
 'Total_Summer',
 'Gold_Winter',
 'Silver_Winter',
 'Bronze_Winter',
 'Total_Winter',
 'Gold_Total',
 'Silver_Total',
 'Bronze_Total',
 'Grand_Total']

In [178]:
[change_type(colname) for colname in lst[1:]]

[None, None, None, None, None, None, None, None, None, None, None, None]

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country        152 non-null    object
 1   Gold_Summer    152 non-null    int64 
 2   Silver_Summer  152 non-null    int64 
 3   Bronze_Summer  152 non-null    int64 
 4   Total_Summer   152 non-null    int64 
 5   Gold_Winter    152 non-null    int64 
 6   Silver_Winter  152 non-null    int64 
 7   Bronze_Winter  152 non-null    int64 
 8   Total_Winter   152 non-null    int64 
 9   Gold_Total     152 non-null    int64 
 10  Silver_Total   152 non-null    int64 
 11  Bronze_Total   152 non-null    int64 
 12  Grand_Total    152 non-null    int64 
dtypes: int64(12), object(1)
memory usage: 15.6+ KB


In [180]:
df['Grand_Total'].sum()

18855

In [181]:
df.to_csv('country_medal_table_clean.csv', index=False)

In [182]:
check = pd.read_csv('country_medal_table_clean.csv')
check.head()

,Country,Gold_Summer,Silver_Summer,Bronze_Summer,Total_Summer,Gold_Winter,Silver_Winter,Bronze_Winter,Total_Winter,Gold_Total,Silver_Total,Bronze_Total,Grand_Total
0,Afghanistan,0,0,2,2,0,0,0,0,0,0,2,2
1,Algeria,5,4,8,17,0,0,0,0,5,4,8,17
2,Argentina,21,25,28,74,0,0,0,0,21,25,28,74
3,Armenia,2,6,6,14,0,0,0,0,2,6,6,14
4,Australasia,3,4,5,12,0,0,0,0,3,4,5,12


In [183]:
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country        152 non-null    object
 1   Gold_Summer    152 non-null    int64 
 2   Silver_Summer  152 non-null    int64 
 3   Bronze_Summer  152 non-null    int64 
 4   Total_Summer   152 non-null    int64 
 5   Gold_Winter    152 non-null    int64 
 6   Silver_Winter  152 non-null    int64 
 7   Bronze_Winter  152 non-null    int64 
 8   Total_Winter   152 non-null    int64 
 9   Gold_Total     152 non-null    int64 
 10  Silver_Total   152 non-null    int64 
 11  Bronze_Total   152 non-null    int64 
 12  Grand_Total    152 non-null    int64 
dtypes: int64(12), object(1)
memory usage: 15.6+ KB
